# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770715766026                   -0.53    8.0         
  2   -2.772139373534       -2.85       -1.31    1.0    162ms
  3   -2.772170239924       -4.51       -2.57    1.0    190ms
  4   -2.772170675239       -6.36       -3.47    1.0    155ms
  5   -2.772170722109       -7.33       -4.02    2.0    216ms
  6   -2.772170722973       -9.06       -4.73    1.0    159ms
  7   -2.772170723011      -10.42       -5.46    1.0    181ms
  8   -2.772170723015      -11.44       -6.17    2.0    188ms
  9   -2.772170723015      -13.42       -6.67    1.0    187ms
 10   -2.772170723015   +  -14.07       -7.03    1.0    173ms
 11   -2.772170723015   +  -14.21       -7.38    1.0    191ms
 12   -2.772170723015      -14.27       -8.47    2.0    201ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735579901989036

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770531789654                   -0.53    9.0         
  2   -2.772048126685       -2.82       -1.30    1.0    150ms
  3   -2.772082596192       -4.46       -2.66    1.0    206ms
  4   -2.772083415075       -6.09       -3.86    2.0    175ms
  5   -2.772083417631       -8.59       -4.40    2.0    229ms
  6   -2.772083417804       -9.76       -5.47    1.0    160ms
  7   -2.772083417811      -11.21       -6.04    2.0    192ms
  8   -2.772083417811      -12.92       -6.96    2.0    186ms
  9   -2.772083417811   +  -13.94       -7.37    1.0    188ms
 10   -2.772083417811   +  -14.31       -8.63    2.0    199ms

Polarizability via ForwardDiff:       1.7725349696871942
Polarizability via finite difference: 1.7735579901989036
